# Model integration

This tutorial demonstrate how an openalea model can be exported as an IPM-compliant web service

## Python Model creation

Let export the folowing simple model, that return some risk level, as a function of air temperature and a user-defined temperature threshold:

In [1]:
def t_risk(tair, threshold=15):
    if tair <= threshold:
        return 0
    else:
        return 1

In [2]:
t_risk(0), t_risk(20), t_risk(20, threshold=20)

(0, 1, 0)

## Import in OpenAlea

Let import it in OpenAlea, by embending it in a OpenAlea Node:

In [3]:
from openalea.core.node import FuncNode
from openalea.core import IFloat, IInt

In [4]:
inputs = (dict(name='tair', interface=IFloat, value=None),
          dict(name='threshold', interface=IFloat, value=15))
outputs = (dict(name='Risk', interface=IInt), )
my_node = FuncNode(inputs, outputs, t_risk)
my_node.name='TRISK'

In [5]:
my_node((0,)), my_node((20,)),my_node((20,20))

(0, 1, 0)

In [6]:
my_node.name, my_node.input_desc, my_node.output_desc

('TRISK',
 [{'name': 'tair', 'interface': IFloat, 'value': None},
  {'name': 'threshold', 'interface': IFloat, 'value': 15}],
 [{'name': 'Risk', 'interface': IInt}])

## Create IPM model

In [8]:
from openalea.dss import Manager
from openalea.dss.dss_factory import dss_factory
help(dss_factory)

Help on function dss_factory in module openalea.dss.dss_factory:

dss_factory(node, interval=86400, weather_parameters=None, parameters=None, decision_support=None, template=None)
    Transform an openalea node in a IPM model json descriptor and generate a fastAPI script to launch webservice
    
    Args:
        node: the node to be exported
        interval: the time step of the model (s)
        weather_parameters: a mapping between node input name and weather data codes, if any.
        None if none of the input is a weather data
        parameters: a list of node input name to be exposed as config parameters in IPM-Decison platform
        template: (optional) an existing IPM model to be used as a template for filling missing information
    Returns:
        model: a json-like dict describing the model
        dss_service: a string containing the script to be run for launching the web service



We will use as a templase a nibio vips model, and define a simple decision_support function that help a user interpret the risk level of our model

In [9]:
decision_support = list(range(2))
decision_support[0] = {'explanation': 'Risk is low',
                      'recommended_action': 'No particular action is required'}
decision_support[1] = {'explanation': 'Risk is high',
                      'recommended_action': 'Be carreful !'}

In [10]:
m = Manager()
template = m.get_model("no.nibio.vips","PSILARTEMP")

In [11]:
ipm_model, service = dss_factory(my_node, weather_parameters={'tair': 1002}, parameters=['threshold'],decision_support=decision_support,template=template)

In [12]:
ipm_model

{'name': 'Carrot fly flight model',
 'id': 'TRISK',
 'version': '1.0',
 'purpose': 'Estimates risk of flight and egg laying in crop',
 'description': 'THE PEST: The first generation of adult carrot fly emerge from pupae in the soil in the spring, and lay eggs close to the base of vulnerable crops. Larvae initial feed at the surface, then tunnel into the tap root. Adults emerge mid-July and can lead to a second generation. \nTHE DECISION: Treatments may need to be applied soon after adults arrive in the crop, before larvae tunnel into the crop roots.  \nTHE MODEL: The model determines the start of the flight period for the 1st generation of carrot rust fly based on accumuleted degree-days (260 day-degrees) over a base temperature of 5°C.  \nTHE PARAMETERS: The model uses daily air temperature \nSOURCE: Luke, Finland. \nASSUMPTIONS: Be aware that in areas with field covers (plastic, single or double non-woven covers, etc.) with early crops the preceding season (either on the current fiel

In [13]:
print(service)

from fastapi import FastAPI
from openalea.dss.dss_factory import encode_input

app = FastAPI()

#hack_import_node (to be done with package manager)
from openalea.core.node import FuncNode
from openalea.core import IFloat, IInt
def t_risk(tair, threshold=15):
    if tair <= threshold:
        return 0
    else:
        return 1
inputs = (dict(name='tair', interface=IFloat, value=None),
          dict(name='threshold', interface=IFloat, value=15))
outputs = (dict(name='Risk', interface=IInt), )
node = FuncNode(inputs, outputs, t_risk)
node.name='TRISK'
# end hack


input_mapping = {'weather_parameters': {"tair": 1002}, 'config_params': ["threshold"]}

@app.post("dss/model/no.nibio.vips/TRISK")
async def model_evaluation(input_data):
    inputs = encode_input(node, input_data, input_mapping)
    return ','.join([str(node(input)) for input in inputs])



## Launch the Webservice

This require to onstall fastapi and uvicorn via pip/conda

In [22]:
with open('main.py', 'w') as outfile:
    outfile.write(service)

Open a terminal in the directory where the file is, activate your python env and lanch the service typing:
    uvicorn main:app

## Test the exported model

In [14]:
from agroservices.ipm.ipm import IPM
from openalea.dss.dss_factory import fake_input_data
import json

In [15]:
ipm = IPM()
input_data = json.loads(fake_input_data)
input_data

{'modelId': 'TRISK',
 'configParameters': {'threshold': 15,
  'timeStart': '2020-05-01',
  'timeEnd': '2020-05-03'},
 'weatherData': {'timeStart': '2020-04-30T22:00:00Z',
  'timeEnd': '2020-05-02T22:00:00Z',
  'interval': 86400,
  'weatherParameters': [1002],
  'locationWeatherData': [{'longitude': 10.781989,
    'latitude': 59.660468,
    'altitude': 94.0,
    'data': [[5.7], [8.2], [8.5]],
    'length': 3,
    'width': 1}]}}

In [23]:
ipm.run_model(ipm_model, fake_input_data)

WARNING [agroservices:IPM:116]:  status is not ok with Unprocessable Entity
WARNING [agroservices:IPM:116]:  status is not ok with Unprocessable Entity


422

In [20]:
ipm_model['execution']['endpoint']

'http://127.0.0.1:8000/dss/model/no.nibio.vips/TRISK'